In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'

In [2]:
# Importing the libraries:
import tensorflow as tf
from tensorflow import keras
import numpy as np
tf.__version__, keras.__version__, tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


('2.10.0', '2.10.0', True)

# Prepping the Images:

In [3]:
# Using a function to automate image loading and transformations:
path = "data/"

data = tf.keras.preprocessing.image_dataset_from_directory(
    path,
    label_mode = None, # To ensure that it doesn't throw me an error and retrieves all the images.
    color_mode = "rgb",
    batch_size = 32,
    image_size = (96, 96),
    shuffle = True,
    
)
data = data.unbatch()
data = data.batch(32, drop_remainder = True)
for i in data:
    i = tf.cast(i, tf.float32)/255.

Found 21551 files belonging to 1 classes.


In [4]:
# Creating some more variables:
coding_size = 128
optimizer = tf.keras.optimizers.Nadam(learning_rate = 0.0005)
loss = tf.keras.losses.binary_crossentropy
batch_size = 32

# Creating a GAN:

In [5]:
# Sequential generator:
generator = keras.models.Sequential([
    keras.layers.Dense(6 * 6 * 1024, input_shape = [coding_size]),
    keras.layers.BatchNormalization(),
    keras.layers.Reshape([6, 6, 1024]),
    keras.layers.Conv2DTranspose(512, kernel_size = 5, strides = 2, padding = "same", activation = 'selu'), # [12, 12, 512]
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(256, kernel_size = 5, strides = 2, padding = "same", activation = 'selu'), # [24, 24, 256]
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(128, kernel_size = 5, strides = 2, padding = "same", activation = 'selu'), # [48, 48, 128]
    keras.layers.BatchNormalization(),
    keras.layers.Conv2DTranspose(3, kernel_size = 5, strides = 2, padding = "same", activation = "tanh") # [96, 96, 3]
])

In [6]:
# Sequential discriminator:
discriminator = keras.models.Sequential([
    keras.layers.Conv2D(64, kernel_size = 5, strides  = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2), input_shape = [96, 96, 3]), # [48, 48, 64]
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, kernel_size = 5, strides = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2)), # [24, 24, 128]
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(256, kernel_size = 3, strides = 2, padding = "same", activation = tf.keras.layers.LeakyReLU(0.2)), # [12, 12, 256]
    keras.layers.Flatten(),
    keras.layers.Dense(1, activation = "sigmoid")
])

In [7]:
# Creating a gan:
gan = keras.models.Sequential([generator, discriminator])

# Training the GAN:

In [8]:
# Creating a image plotter:
def plot(x_train, image_no = 1):
    for i in tf.range(image_no):
        img = x_train[i]
        img = tf.keras.preprocessing.image.array_to_img(img)
        img.show()
        img.save("Progress/")


In [9]:
# Creating an custom callback:
def Checkpoint(model):
    model.save("Checkpoints/GAN.h5")

In [10]:
# General Step:
def train_step(model, x, y, loss_fn = loss, optimizer = optimizer):
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y_pred = y_pred, y_true = y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [11]:
#  Discriminator step:
@tf.function
def train_d(model, noise, x_batch, batch_size = batch_size):
    generator, discriminator = model.layers
    predictions = generator(noise)
    inputs = tf.concat([predictions, x_batch], axis = 0)
    y = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
    discriminator.trainable = True
    train_step(discriminator, inputs, y)

In [12]:
# Generator step:
@tf.function
def train_g(model, noise, batch_size = batch_size):
    discriminator = model.layers[1]
    discriminator.trainable = False
    y = tf.constant([[1.]] * batch_size)
    train_step(model, noise, y)

In [13]:
# Predicting values:
def predict(model, batch_size = batch_size, coding_size = coding_size):
    noise = tf.random.normal([batch_size, coding_size])
    generator = model.layers[0]
    predictions = generator(noise)
    plot(predictions)

In [14]:
# Training Loop:
epochs = 20
for epoch in range(epochs):
    print(f"Epochs {epoch + 1} started.")
    for x_batch in data:
        # Training the Discriminator:
        noise = tf.random.normal([batch_size, coding_size])
        train_d(gan, noise, x_batch)
        # Training the Generator:
        noise = tf.random.normal([batch_size, coding_size])
        train_g(gan, noise)
    predict(gan)
    Checkpoint(gan)

Epochs 1 started.
Epochs 2 started.



(eog:12546): Gtk-WARNING **: 12:39:50.371: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.

(eog:12546): Gtk-WARNING **: 12:39:51.442: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)


Epochs 3 started.



(eog:12669): Gtk-WARNING **: 12:40:35.257: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.

(eog:12669): Gtk-WARNING **: 12:40:36.774: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)


Epochs 4 started.
Epochs 5 started.
Epochs 6 started.



(eog:12965): Gtk-WARNING **: 12:43:24.592: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.

(eog:12965): Gtk-WARNING **: 12:43:25.857: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)


Epochs 7 started.



(eog:13086): Gtk-WARNING **: 12:43:43.025: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.

(eog:13086): Gtk-WARNING **: 12:43:44.669: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)


Epochs 8 started.
Epochs 9 started.



(eog:13171): Gtk-WARNING **: 12:45:18.165: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.

(eog:13171): Gtk-WARNING **: 12:45:19.081: Negative content width -1 (allocation 23, extents 12x12) while allocating gadget (node label, owner GtkLabel)
